# Deep Learning Models

In [1]:
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics
from sklearn.metrics import accuracy_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

import matplotlib.pyplot as plt

In [2]:
Final_df = pd.read_csv('./Clean_dataframe/clean_df.csv')

In [3]:
features = ['LONGITUDE', 'ELEVATION','TMIN', 'TMAX' ,'PRCP','TAVG']
X = Final_df[features]
y = Final_df['sea_level']

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

In [4]:
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.fit_transform(X_test)

In [5]:
# construct network
model = Sequential()

# first layer
model.add(Dense(32, input_shape=(6,), activation='relu'))

# second layer
model.add(Dense(32, activation='relu'))

# output layer
model.add(Dense(1))

In [6]:
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [7]:
model.fit(X_train_sc, y_train, validation_data=(X_test_sc, y_test), epochs=10, batch_size=512);

Epoch 1/10
139/139 [==============================] - 1s 2ms/step - loss: 7945449.5000 - mae: 2630.9585 - val_loss: 7882499.5000 - val_mae: 2617.6626
Epoch 2/10
139/139 [==============================] - 0s 859us/step - loss: 7596492.0000 - mae: 2566.2830 - val_loss: 7192419.0000 - val_mae: 2490.0212
Epoch 3/10
139/139 [==============================] - 0s 830us/step - loss: 6442424.5000 - mae: 2345.4912 - val_loss: 5568791.5000 - val_mae: 2166.2920
Epoch 4/10
139/139 [==============================] - 0s 823us/step - loss: 4539192.5000 - mae: 1932.9907 - val_loss: 3545583.0000 - val_mae: 1684.2080
Epoch 5/10
139/139 [==============================] - 0s 829us/step - loss: 2750986.5000 - mae: 1446.9980 - val_loss: 2128283.0000 - val_mae: 1235.9755
Epoch 6/10
139/139 [==============================] - 0s 832us/step - loss: 1751445.6250 - mae: 1082.2332 - val_loss: 1489847.6250 - val_mae: 965.1890
Epoch 7/10
139/139 [==============================] - 0s 839us/step - loss: 1301110.6250 - 

In [8]:
model.predict(X_test_sc)

array([[2091.5767],
       [2487.441 ],
       [1629.4806],
       ...,
       [ 969.9026],
       [1988.1176],
       [2513.329 ]], dtype=float32)

In [9]:
model.evaluate(X_test_sc, y_test, batch_size=512)

47/47 [==============================] - 0s 521us/step - loss: 676209.0625 - mae: 558.7137


[676209.0625, 558.7137451171875]

## GridSearch

In [10]:
def model_fn_adv(hidden_neurons = 32, hidden_layers = 5, dropout = 0.5):
    # build framework of model using for loop and if statement
    model = Sequential()
    
    # adding number of layers = specified in calling of function
    for layer in range(hidden_layers):
        if layer == 0:
            model.add(Dense(hidden_neurons, activation = 'relu', input_shape = (6,)))
            model.add(Dropout(dropout))
        else:
            model.add(Dense(hidden_neurons, activation = 'relu'))
            model.add(Dropout(dropout))
    
    # out put layer added here, Identity link function used as 'None'
    model.add(Dense(1, activation = None))

    # using mse as the metric to solve for 
    model.compile(loss = 'mse', optimizer = 'adam')
    
    return model

In [11]:
nn = KerasRegressor(build_fn = model_fn_adv, batch_size = 512, verbose = 0)

In [12]:
nn_params_deep = {
    'hidden_neurons' : [20, 32, 64],
    'hidden_layers'  : [2, 3, 5],
    'dropout'        : [0.1, 0.2, 0.3, 0.4, 0.5],
    'epochs'         : [10, 20, 30, 40, 50]
}

In [13]:
gs = GridSearchCV(nn, param_grid = nn_params_deep, cv = 5)

In [14]:
gs.fit(X_train_sc, y_train)

GridSearchCV(cv=5,
             estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7fea87b10190>,
             param_grid={'dropout': [0.1, 0.2, 0.3, 0.4, 0.5],
                         'epochs': [10, 20, 30, 40, 50],
                         'hidden_layers': [2, 3, 5],
                         'hidden_neurons': [20, 32, 64]})

In [15]:
gs.best_params_

{'dropout': 0.1, 'epochs': 10, 'hidden_layers': 5, 'hidden_neurons': 64}

In [16]:
gs.best_score_

-43213.00625

In [17]:
# Make predictions 
preds = gs.predict(X_test_sc)

In [20]:
# Check the R-squared
metrics.r2_score(y_test, preds)

0.9291870651738445

## Recurrent Neural Networks

In [ ]:
features = ['LONGITUDE', 'ELEVATION','TMIN', 'TMAX' ,'PRCP','TAVG']
X = Final_df[features]
y = Final_df['sea_level']

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

In [ ]:
ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

In [ ]:
train_sequences = TimeseriesGenerator(X_train_sc, y_train, length=40, batch_size=64)

In [ ]:
test_sequences = TimeseriesGenerator(X_test_sc, y_test, length=40, batch_size=64)

In [ ]:
input_shape = train_sequences[0][0][0].shape

In [ ]:
model = Sequential()
model.add(GRU(8, input_shape=input_shape, return_sequences=True)) # True if next layer is RNN
model.add(GRU(8, return_sequences=False)) # False if next layer is Dense

model.add(Dense(4, activation='relu'))

model.add(Dense(1))

In [ ]:
model.compile(optimizer='adam', loss='bce', metrics=['acc'])

history = model.fit(train_sequences, validation_data=test_sequences, epochs=50, verbose=0)

In [ ]:
plt.plot(history.history['loss'], label='Train loss')
plt.plot(history.history['val_loss'], label='Test loss')
plt.legend();

In [ ]:
plt.plot(history.history['acc'], label='Train accuracy')
plt.plot(history.history['val_acc'], label='Test accuracy')
plt.legend();

In [ ]:
# Add early stopping
from tensorflow.keras.callbacks import EarlyStopping

model2 = Sequential()
model2.add(GRU(8, input_shape=(3, 8), return_sequences=True)) # True if next layer is RNN
model2.add(GRU(8, return_sequences=False)) # False if next layer is Dense

model2.add(Dense(4, activation='relu'))

model2.add(Dense(1))

model2.compile(optimizer='adam', loss='bce', metrics=['acc'])

es = EarlyStopping(patience = 5)

history2 = model2.fit(train_sequences, validation_data=test_sequences, epochs=100, batch_size = 64, callbacks = [es])

In [ ]:
plt.plot(history2.history['loss'], label='Train loss')
plt.plot(history2.history['val_loss'], label='Test loss')
plt.plot(history.history['loss'], label='Original Train loss')
plt.plot(history.history['val_loss'], label='Original Test loss')
plt.legend();